# **Multigrams: Full Workflow**
The multigram workflow mirrors the unigram workflow, but with two differences. First, instead of _creating_ a whitelist, you filter the multigram corpus _using_ a whitelist containing the top-_N_ unigrams. Second, the multigram workflow adds a **pivoting** step. Pivoting reorganizes the database so that it's easy to query year-ngram combinations. For instance, you can learn how many times the word "nuclear" appeared in 2011 by querying the key `[2011] nuclear`. This is useful for analyzing changes in word meanings over time.

## **Setup**
### Imports

In [1]:
%load_ext autoreload
%autoreload 2

from ngramkit.ngram_acquire import download_and_ingest_to_rocksdb
from ngramkit.ngram_filter.pipeline.orchestrator import build_processed_db
from ngramkit.ngram_pivot.pipeline import build_pivoted_db
from ngramkit.utilities.peek import db_head, db_peek, db_peek_prefix

### Configure
Here we set basic parameters: the corpus to download, the size of the ngrams to download, and the size of the year bins.

In [2]:
db_path_stub = '/scratch/edk202/NLP_corpora/Google_Books/'
archive_path_stub = '/scratch/edk202/NLP_archive/Google_Books/'
release = '20200217'
language = 'eng'
ngram_size = 5
bin_size = 5

## **Step 1: Download and Ingest**

In [3]:
combined_bigrams_download = {"lower class", "working class", "middle class", "upper class", "blue collar", "white collar", "african american", "african americans", "european american", "european americans", "white people", "white person", "white americans", "black american", "black americans", "black person", "black people", "human being"}

download_and_ingest_to_rocksdb(
    ngram_size=size,
    repo_release_id=release,
    repo_corpus_id=language,
    db_path_stub=db_path_stub,
    archive_path_stub=None,
    ngram_type="tagged",
    random_seed=98,
    overwrite_db=False,
    workers=80,
    write_batch_size=5_000,
    open_type="write:packed24",
    compact_after_ingest=True,
    combined_bigrams=combined_bigrams_download
)

N-GRAM ACQUISITION PIPELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start Time: 2025-11-23 20:40:01

Download Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Ngram repo:           https://books.storage.googleapis.com/?prefix=ngrams/books/20200217/eng/5-
DB path:              /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db
File range:           0 to 19422
Total files:          19423
Files to get:         0
Skipping:             19423
Download workers:     80
Batch size:           5,000
Ngram size:           5
Ngram type:           tagged
Overwrite DB:         False
DB Profile:           write:packed24

Download Progress
════════════════════════════════════════════════════════════════════════════════════════════════════


Files Processed:   0%|                                                               | 0/0 [00:00<?]


Post-Ingestion Compaction
════════════════════════════════════════════════════════════════════════════════════════════════════
Initial DB size:         2.21 TB


Compaction completed in 0:42:07
Size before:             2.21 TB
Size after:              2.21 TB
Space saved:             -143.08 KB (-0.0%)

Processing complete!

Final Summary
════════════════════════════════════════════════════════════════════════════════════════════════════
Fully processed files:       0
Failed files:                0
Total entries written:       0
Write batches flushed:       0
Uncompressed data processed: 0.00 B
Processing throughput:       0.00 MB/sec

End Time: 2025-11-23 21:22:26.459660
Total Runtime: 0:42:25.150204
Time per file: 0:00:00
Files per hour: 0.0


## **Step 2: Filter and Normalize**
`config.py` contains generic defaults for the filtering pipeline. You can override these defaults by passing option dictionaries to the `build_processed_db` function, as seen below. As implemented here, we use the whitelist from the unigram workflow to filter the multigram corpus. If we weren't using a whitelist, we could normalize, filter, and lemmatize each token just as we did for the unigrams.

In [3]:
filter_options = {
    'bin_size': bin_size
}

whitelist_options = {
    'whitelist_path': f'{db_path_stub}/{release}/{language}/1gram_files/1grams_processed.db/whitelist.txt',
    'output_whitelist_path': None
}

always_include_tokens = {"lower-class", "working-class", "middle-class", "upper-class", "blue-collar", "white-collar", "african-american", "african-americans", "european-american", "european-americans", "white-people", "white-person", "white-americans", "black-american", "black-americans", "black-person", "black-people", "human-being"}

build_processed_db(
    mode="restart",
    ngram_size=ngram_size,
    repo_release_id=release,
    repo_corpus_id=language,
    db_path_stub=db_path_stub,
    num_workers=36,
    num_initial_work_units=600,
    work_unit_claim_order="random",
    cache_partitions=True,
    use_cached_partitions=True,
    progress_every_s=30.0,
    always_include=always_include_tokens,
    **filter_options,
    **whitelist_options
);


N-GRAM FILTER PIPELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start Time: 2025-11-29 19:16:52
Mode:       RESTART

Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Source DB:            /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db
Target DB:            ...dk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db
Temp directory:       ...tch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/processing_tmp

Parallelism
────────────────────────────────────────────────────────────────────────────────────────────────────
Workers:              36
Initial work units:   600

Database Profiles
────────────────────────────────────────────────────────────────────────────────────────────────────
Reader profile:       read:packed24
Writer profile:       write:packed24

Ingestion Configuration
────────────────────────

Shards Ingested: 100%|███████████████████████████████████████████████████████| 600/600 [26:29<00:00]



Ingestion complete: 600 shards, 501,124,594 items in 1590.0s (315,176 items/s)

Phase 4: Finalizing database...
════════════════════════════════════════════════════════════════════════════════════════════════════

Post-Ingestion Compaction
────────────────────────────────────────────────────────────────────────────────────────────────────
Initial DB size:         181.05 GB
Compaction completed in 0:04:22
Size before:             181.05 GB
Size after:              115.49 GB
Space saved:             65.56 GB (36.2%)

┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
│ PROCESSING COMPLETE                                                                              │
├──────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Items: 438,496,183 (estimated)                                                                   │
│ Size: 115.49 GB                                                         

## **Step 3: Pivot to Yearly Indices**

In [4]:
build_pivoted_db(
    mode="restart",
    ngram_size=ngram_size,
    repo_release_id=release,
    repo_corpus_id=language,
    db_path_stub=db_path_stub,
    num_workers=40,
    num_initial_work_units=600,
    cache_partitions=True,
    use_cached_partitions=True,
    progress_every_s=30.0,
);


PARALLEL N-GRAM DATABASE PIVOT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start Time: 2025-11-29 20:21:33
Mode:       RESTART

Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Source DB:            ...dk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db
Target DB:            .../edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_pivoted.db
Temp directory:       /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/pivoting_tmp

Parallelism
────────────────────────────────────────────────────────────────────────────────────────────────────
Workers:              40
Initial work units:   600

Database Profiles
────────────────────────────────────────────────────────────────────────────────────────────────────
Reader profile:       read:packed24
Writer profile:       write:packed24
Ingest profile:       write:packed24



SST Files Ingested: 100%|████████████████████████████████████████████████████| 600/600 [00:29<00:00]



Phase 4: Finalizing database...
════════════════════════════════════════════════════════════════════════════════════════════════════

Post-Ingestion Compaction
────────────────────────────────────────────────────────────────────────────────────────────────────
Initial DB size:         72.78 GB
Compaction completed in 0:11:46
Size before:             72.78 GB
Size after:              172.46 GB
Space saved:             -99.68 GB (-137.0%)

┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
│ PROCESSING COMPLETE                                                                              │
├──────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Items: 4,896,078,077 (estimated)                                                                 │
│ Size: 172.46 GB                                                                                  │
│ Database: /scratch/edk202/NLP_corpora/Google_Books

# Inspect Final Database
Here are three functions you can use to inspect the final database.

## `db_head`: First N records

In [6]:
pivoted_db = f'{db_path_stub}{release}/{language}/{ngram_size}gram_files/{ngram_size}grams_pivoted.db'

db_head(pivoted_db, n=5)

First 5 key-value pairs:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [1470] <UNK> <UNK> <UNK> <UNK> absence
     Value: 5 occurrences in 3 documents

[ 2] Key:   [1470] <UNK> <UNK> <UNK> <UNK> accordance
     Value: 1 occurrences in 1 documents

[ 3] Key:   [1470] <UNK> <UNK> <UNK> <UNK> accordingly
     Value: 1 occurrences in 1 documents

[ 4] Key:   [1470] <UNK> <UNK> <UNK> <UNK> accrue
     Value: 1 occurrences in 1 documents

[ 5] Key:   [1470] <UNK> <UNK> <UNK> <UNK> acre
     Value: 2 occurrences in 2 documents



## `db_peek`: Records starting from a key

In [15]:
pivoted_db = f'{db_path_stub}{release}/{language}/{ngram_size}gram_files/{ngram_size}grams_pivoted.db'

db_peek(pivoted_db, start_key="[2015] working-class <UNK> <UNK> <UNK> <UNK>", n=5)

5 key-value pairs starting from 000007df776f726b696e672d636c617373203c554e4b3e203c554e4b3e203c554e4b3e203c554e4b3e:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [2015] working-class <UNK> <UNK> ability
     Value: 13 occurrences in 13 documents

[ 2] Key:   [2015] working-class <UNK> <UNK> able
     Value: 120 occurrences in 117 documents

[ 3] Key:   [2015] working-class <UNK> <UNK> abolition
     Value: 45 occurrences in 42 documents

[ 4] Key:   [2015] working-class <UNK> <UNK> abortive
     Value: 2 occurrences in 2 documents

[ 5] Key:   [2015] working-class <UNK> <UNK> absence
     Value: 13 occurrences in 13 documents



## `db_peek_prefix`: Records matching a prefix

In [16]:
pivoted_db = f'{db_path_stub}{release}/{language}/{ngram_size}gram_files/{ngram_size}grams_pivoted.db'

db_peek_prefix(pivoted_db, prefix="[2000] <UNK> working-class", n=5)

5 key-value pairs with prefix 000007d03c554e4b3e20776f726b696e672d636c617373:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [2000] <UNK> working-class <UNK> <UNK>
     Value: 138,742 occurrences in 128,220 documents

[ 2] Key:   [2000] <UNK> working-class <UNK> ability
     Value: 3 occurrences in 3 documents

[ 3] Key:   [2000] <UNK> working-class <UNK> able
     Value: 108 occurrences in 107 documents

[ 4] Key:   [2000] <UNK> working-class <UNK> absent
     Value: 5 occurrences in 5 documents

[ 5] Key:   [2000] <UNK> working-class <UNK> absolutely
     Value: 5 occurrences in 5 documents

